In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
books = pd.read_csv(r'dataset/books.csv')
ratings = pd.read_csv(r'dataset/ratings.csv')
book_tags = pd.read_csv(r'dataset/book_tags.csv')
tags = pd.read_csv(r'dataset/tags.csv')

# Item based SVD

In [3]:
ratings=ratings.drop_duplicates()

In [4]:
ratings.shape

(5976479, 3)

In [5]:
unwanted_users = ratings.groupby('user_id')['user_id'].count()
unwanted_users = unwanted_users[unwanted_users < 30]

In [6]:
unwanted_rating=ratings[ratings.user_id.isin(unwanted_users.index)]

In [7]:
ratings.drop(unwanted_rating.index,inplace=True)

In [8]:
books=books.dropna(subset=['original_publication_year'])
books=books.dropna(subset=['title'])

In [9]:
book_name=books[['book_id','title']]

In [10]:
book_name=book_name.reset_index(drop=True)

In [11]:
new_rating=pd.merge(ratings,book_name,how='inner',on='book_id',sort=True)

In [12]:
new_rating=new_rating.sort_values(by='user_id')

In [13]:
new_rating=new_rating.dropna(axis=0)

In [14]:
new_rating

,user_id,book_id,rating,title
2176658,1,421,5,The Paris Wife
2963390,1,869,4,The Elegance of the Hedgehog
166325,1,11,5,The Kite Runner
1407603,1,179,2,"Angela's Ashes (Frank McCourt, #1)"
847103,1,76,3,Sense and Sensibility
...,...,...,...,...
2235620,53424,447,3,"Brisingr (The Inheritance Cycle, #3)"
1391264,53424,175,5,The Last Olympian (Percy Jackson and the Olymp...
1337014,53424,163,4,"The Lost Hero (The Heroes of Olympus, #1)"
1967327,53424,340,4,"I Am Number Four (Lorien Legacies, #1)"


In [15]:
piv_tab=new_rating.pivot_table(index='user_id',columns='book_id',values='rating', fill_value=0)

In [16]:
piv_tab

book_id,1,2,3,4,5,6,7,8,9,10,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,5,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
2,0,5,0,0,5,0,0,4,0,5,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,5,0,4,4,0,4,4,0,5,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53420,4,5,3,0,2,0,0,0,4,3,...,0,0,0,0,0,0,0,0,0,0
53421,4,5,0,5,4,0,4,0,5,0,...,0,0,0,0,0,0,0,0,0,0
53422,4,5,0,0,0,0,5,0,0,5,...,0,0,0,0,0,0,0,0,0,0


In [17]:
import sklearn
from sklearn.decomposition import TruncatedSVD

In [18]:
X = piv_tab.T

In [19]:
X1=X

In [20]:
SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

(9979, 10)

In [21]:
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

(9979, 9979)

In [22]:
correlation_matrix

array([[1.        , 0.7963354 , 0.87011024, ..., 0.41688085, 0.57127662,
        0.45462582],
       [0.7963354 , 1.        , 0.72221368, ..., 0.55310334, 0.40310033,
        0.47044062],
       [0.87011024, 0.72221368, 1.        , ..., 0.15737375, 0.29453331,
        0.14026702],
       ...,
       [0.41688085, 0.55310334, 0.15737375, ..., 1.        , 0.37687732,
        0.81515354],
       [0.57127662, 0.40310033, 0.29453331, ..., 0.37687732, 1.        ,
        0.59413811],
       [0.45462582, 0.47044062, 0.14026702, ..., 0.81515354, 0.59413811,
        1.        ]])

In [23]:
#Twilight (Twilight, #1)
correlation_product_ID=correlation_matrix[4]

In [24]:
#out_put is book_id 
Recommend = list(X.index[correlation_product_ID > 0.90])

# Removes the item already bought by the customer

In [25]:
Recommend

[4,
 5,
 8,
 13,
 14,
 15,
 28,
 29,
 32,
 35,
 45,
 48,
 55,
 58,
 60,
 71,
 79,
 80,
 82,
 83,
 87,
 94,
 95,
 113,
 119,
 121,
 122,
 127,
 129,
 130,
 131,
 138,
 160,
 172,
 177,
 178,
 180,
 181,
 193,
 194,
 198,
 211,
 212,
 225,
 233,
 255,
 264,
 287,
 301,
 354,
 357,
 395,
 397,
 401,
 410,
 427,
 456,
 462,
 468,
 472,
 485,
 489,
 560,
 580,
 595,
 597,
 635,
 664,
 669,
 690,
 701,
 718,
 730,
 736,
 763,
 765,
 782,
 795,
 819,
 879,
 899,
 1084,
 1161,
 1184,
 1214,
 1272,
 1393,
 1411,
 1449,
 1509,
 1518,
 1566,
 1595,
 1664,
 1681,
 1691,
 1777,
 1785,
 1973,
 2022,
 2052,
 2053,
 2108,
 2131,
 2145,
 2157,
 2190,
 2197,
 2257,
 2308,
 2314,
 2336,
 2380,
 2403,
 2419,
 2586,
 2601,
 2720,
 2724,
 2729,
 2747,
 2797,
 2875,
 2921,
 2934,
 3002,
 3076,
 3181,
 3210,
 3349,
 3381,
 3387,
 3388,
 3472,
 3511,
 3518,
 3523,
 3535,
 3676,
 3734,
 3763,
 3853,
 3916,
 3939,
 4091,
 4149,
 4244,
 4284,
 4298,
 4359,
 4597,
 4777,
 4780,
 4785,
 4865,
 4974,
 5198,
 5237,
 

In [26]:
book_name.index=book_name.book_id

In [27]:
book_name.loc[Recommend]

,book_id,title
book_id,,
4,4,To Kill a Mockingbird
5,5,The Great Gatsby
8,8,The Catcher in the Rye
13,13,1984
14,14,Animal Farm
...,...,...
9344,9344,Social Intelligence: The New Science of Human ...
9620,9620,The Complete Poems 1927-1979
9623,9623,"Look Homeward, Angel"
